<a href="https://colab.research.google.com/github/nafiulaziz/multilingualRag/blob/main/rag_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 2: Install dependencies

In [2]:
!pip install pdfplumber sentence-transformers faiss-cpu transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

# Step 3: Import libs

In [3]:
import pdfplumber
import re
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Step 4: Extract text from PDF

In [4]:
def extract_text_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

pdf_path = "/content/drive/My Drive/HSC26_Bangla_1st_Paper.pdf"
raw_text = extract_text_pdfplumber(pdf_path)

print("Extracted Text Sample:\n", raw_text[:1000])

Extracted Text Sample:
 িাাং া
১ম পত্র
আল ািয রিষয়
অপরিরিতা
অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,
র্িখনফল
✓ র্নম্নর্ব্িব্যজক্তিহঠাৎর্ব্িিালীহক ওঠািফকলসমাকিপর্িচ সংকটসম্পককি ধািণালাভকিকব্।
✓ তৎকালীনসমাি-সভযতাওমানব্তািঅব্মাননাসম্পককি িানকতপািকব্।
✓ তৎকালীনসমাকিিপণপ্রথািকুপ্রভাব্সম্পককি িানকতপািকব্।
✓ তৎকাকলসমাকিভদ্রকলাককিস্বভাব্বব্র্িষ্ট্যসম্পককি জ্ঞাানলাভকিকব্।
✓ নািী যকামল ঠিক, র্কন্তু দব্ু লি ন - কলযাণীি িীব্নচর্িত দ্বািা প্রর্তজিত এই সতয অনুধাব্ন কিকত
পািকব্।
✓ মানুষ আিা র্নক যব্কঁ চ থাকক- অনপু কমি দষ্ট্ৃ াকন্ত মানব্িীব্কনি এই র্চিন্তন সতযদিিন সম্পককি
জ্ঞাানলাভকিকব্।
প্রাক-মূলযা ন
১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?
ক) ডাক্তার্ি খ) ওকালর্ত গ) মাস্টার্ি ঘ) ব্যব্সা
২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি-
ক) প্রর্তপজি খ) প্রভাব্ গ) র্ব্চক্ষণতা ঘ) কূট ব্ুর্ি
র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও।
র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা
তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক ব্া়োব্ার়্ে 

# Step 5: Basic cleaning function (remove extra whitespace, keep Bengali & English chars)

In [5]:
def clean_text(text):
    # Keep Bengali letters, English letters, digits, basic punctuation, and spaces
    allowed_chars = re.compile(r'[^\u0980-\u09FFa-zA-Z0-9\s.,;:\'\"?-]')
    text = allowed_chars.sub('', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

cleaned_text = clean_text(raw_text)
print("\nCleaned Text Sample:\n", cleaned_text[:1000])


Cleaned Text Sample:
 িাাং া ১ম পত্র আল ািয রিষয় অপরিরিতা অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা , র্িখনফল র্নম্নর্ব্িব্যজক্তিহঠাৎর্ব্িিালীহক ওঠািফকলসমাকিপর্িচ সংকটসম্পককি ধািণালাভকিকব্ তৎকালীনসমাি-সভযতাওমানব্তািঅব্মাননাসম্পককি িানকতপািকব্ তৎকালীনসমাকিিপণপ্রথািকুপ্রভাব্সম্পককি িানকতপািকব্ তৎকাকলসমাকিভদ্রকলাককিস্বভাব্বব্র্িষ্ট্যসম্পককি জ্ঞাানলাভকিকব্ নািী যকামল ঠিক, র্কন্তু দব্ু লি ন - কলযাণীি িীব্নচর্িত দ্বািা প্রর্তজিত এই সতয অনুধাব্ন কিকত পািকব্ মানুষ আিা র্নক যব্কঁ চ থাকক- অনপু কমি দষ্ট্ৃ াকন্ত মানব্িীব্কনি এই র্চিন্তন সতযদিিন সম্পককি জ্ঞাানলাভকিকব্ প্রাক-মূলযা ন ১ অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন? ক ডাক্তার্ি খ ওকালর্ত গ মাস্টার্ি ঘ ব্যব্সা ২ োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি- ক প্রর্তপজি খ প্রভাব্ গ র্ব্চক্ষণতা ঘ কূট ব্ুর্ি র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না চাচা তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক ব্া়োব্ার়্ে কিাি কািকণ কনযাি র্পতা অপমার্নত 

# Step 6: Chunking the text (e.g. 300 tokens approx)

In [6]:
def chunk_text(text, max_len=300):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_len):
        chunk = ' '.join(words[i:i+max_len])
        chunks.append(chunk)
    return chunks

chunks = chunk_text(cleaned_text, max_len=300)
print(f"\nNumber of chunks: {len(chunks)}")
print("Sample chunk:\n", chunks[0])


Number of chunks: 38
Sample chunk:
 িাাং া ১ম পত্র আল ািয রিষয় অপরিরিতা অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা , র্িখনফল র্নম্নর্ব্িব্যজক্তিহঠাৎর্ব্িিালীহক ওঠািফকলসমাকিপর্িচ সংকটসম্পককি ধািণালাভকিকব্ তৎকালীনসমাি-সভযতাওমানব্তািঅব্মাননাসম্পককি িানকতপািকব্ তৎকালীনসমাকিিপণপ্রথািকুপ্রভাব্সম্পককি িানকতপািকব্ তৎকাকলসমাকিভদ্রকলাককিস্বভাব্বব্র্িষ্ট্যসম্পককি জ্ঞাানলাভকিকব্ নািী যকামল ঠিক, র্কন্তু দব্ু লি ন - কলযাণীি িীব্নচর্িত দ্বািা প্রর্তজিত এই সতয অনুধাব্ন কিকত পািকব্ মানুষ আিা র্নক যব্কঁ চ থাকক- অনপু কমি দষ্ট্ৃ াকন্ত মানব্িীব্কনি এই র্চিন্তন সতযদিিন সম্পককি জ্ঞাানলাভকিকব্ প্রাক-মূলযা ন ১ অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন? ক ডাক্তার্ি খ ওকালর্ত গ মাস্টার্ি ঘ ব্যব্সা ২ োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি- ক প্রর্তপজি খ প্রভাব্ গ র্ব্চক্ষণতা ঘ কূট ব্ুর্ি র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না চাচা তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক ব্া়োব্ার়্ে কিাি কািকণ কনযাি র

# Step 7: Load multilingual embedding model (sentence-transformers)

In [7]:
embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Step 8: Create embeddings for chunks

In [8]:
chunk_embeddings = embedder.encode(chunks, show_progress_bar=True)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

# Step 9: Build FAISS index for similarity search

In [9]:
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)

# Step 10: Load a local causal LM for answer generation (small model for demo)

In [11]:
model_name = "facebook/opt-1.3b"  # Change if you want smaller/larger models
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


# Step 11: Define query processing & retrieval

In [12]:
def retrieve_relevant_chunks(query, k=3):
    query_emb = embedder.encode([query])
    distances, indices = index.search(query_emb, k)
    retrieved_chunks = [chunks[i] for i in indices[0]]
    return retrieved_chunks

# Step 12: Generate answer grounded on retrieved text

In [13]:
def generate_answer(query):
    retrieved = retrieve_relevant_chunks(query)
    context = "\n".join(retrieved)
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    outputs = generator(prompt, max_length=256, num_return_sequences=1)
    answer = outputs[0]['generated_text'].split('Answer:')[-1].strip()
    return answer

# Step 13: Test with English and Bangla queries

In [14]:
english_query = "What is the importance of language in culture?"
bangla_query = "ভাষার গুরুত্ব কি?"

print("\nEnglish Query:", english_query)
print("Answer:", generate_answer(english_query))

print("\nBangla Query:", bangla_query)
print("Answer:", generate_answer(bangla_query))


English Query: What is the importance of language in culture?


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=257) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: রী রাোজ নড় বিখেট পুল্প্রবিয় পুলিপ্রবিয় সৈস্থারাল রামারাল যাজীজীযাতের শর্ষিয় সাফের রামারাল আরংতীযাতের মুবারাল করামারাল যাজ�

Bangla Query: ভাষার গুরুত্ব কি?


Both `max_new_tokens` (=256) and `max_length`(=257) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: ়ানার্জিয়াংটিয়ের অনুপকমি খত্রায়াংটিয়ের গ্রিয়াংটিয়ের সংোণায়াংটি অনুপকমি বেরীপাংটি গ্রিয়াংটি বিংধের কার্থান। টাইম শের কা


#Step 14: UI

In [15]:
!pip install gradio --quiet

import gradio as gr

def answer_interface(query):
    return generate_answer(query)

iface = gr.Interface(fn=answer_interface,
                     inputs=gr.Textbox(lines=2, placeholder="Type question in Bangla or English"),
                     outputs="text",
                     title="Multilingual RAG Q&A")

iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60ac43b30e8a6ede7f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
